In [1]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import random
import wandb
import copy
import time
from datetime import datetime
from pytz import timezone
import seaborn as sns
import torch.distributions.normal.Normal as Normal
import cv2

from torchvision.transforms import ToTensor,Compose,ColorJitter,RandomRotation,RandomHorizontalFlip,Resize,CenterCrop,Normalize,GaussianBlur,RandomCrop

import timm

import torch
import torch.nn as nn
import torch.optim as optm
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import torchvision

from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from matplotlib import gridspec

from torchinfo import summary
from tqdm.auto import tqdm

%matplotlib inline
device = 'cuda' if torch.cuda.is_available() else 'cpu'
cwd=os.path.dirname(os.getcwd())
KST=timezone('Asia/Seoul')

In [3]:
#프로젝트 이름 설정과 저장경로
project_name='BestEachClassEfficientNetTest'
target_name='EachClassEfficientNet'
target_dir=os.path.join(os.getcwd(),f'saved/{target_name}/model')
model_dir=os.path.join(os.getcwd(),f'saved/{project_name}/model')
checkpoint_dir=os.path.join(os.getcwd(),f'saved/{project_name}/checkpoint')
result_dir=os.path.join(os.path.dirname(checkpoint_dir),'result')
wandb_dir=os.path.dirname(checkpoint_dir)
if not os.path.isdir(checkpoint_dir):
    os.makedirs(checkpoint_dir)
if not os.path.isdir(result_dir):
    os.makedirs(result_dir)
if not os.path.isdir(model_dir):
    os.makedirs(model_dir)

In [4]:
#Randomness 제어
random_seed=2021
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

In [43]:
transform={'trans':Compose([
    Resize((256*512//384,256)),
    CenterCrop(224),
    ToTensor(),
    Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
]),'regtrans':Compose([
    GaussianBlur(3),
    ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    Resize((256*512//384,256)),
    RandomCrop(224),
    ToTensor(),
    Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
]),
'uptrans':Compose([
    ColorJitter(brightness=0.25, contrast=0),
    RandomHorizontalFlip(),
    RandomRotation(20),
    Resize((256*512//384,256)),
    CenterCrop(224),
    ToTensor(),
    Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
])}

def make_images(meta,img_dir,train):
    images=[]
    labels=[]
    data=pd.DataFrame({'images':[],'labels':[]})
    if train:
        for idx in range(len(meta)):
            folder_path=os.path.join(img_dir, meta.path.iloc[idx])
            for img in os.listdir(folder_path):
                if '._' in img:
                    continue
                images.append(os.path.join(folder_path,img))
                labels.append((('incorrect' in img)+('normal' in img)*2)*6+(meta.gender.iloc[idx]=='female')*3+(30<=meta.age.iloc[idx])+(60<=meta.age.iloc[idx]))
    else:
        for img_id in meta.ImageID:
            images.append(os.path.join(img_dir, img_id))
        labels=[0 for _ in range(len(images))]
    data['images']=images
    data['labels']=labels
    return data

class ImageDataset(Dataset):
    def __init__(self,transform=transform,mod='train'):
        self.mod=mod
        self.train=mod!='test'
        self.md=['info','train']
        self.path=[os.path.join(cwd,'input/data/eval'),os.path.join(cwd,'input/data/train')]
        self.meta=pd.read_csv(os.path.join(self.path[self.train], f'{self.md[self.train]}.csv'))
        self.img_dir=os.path.join(self.path[self.train],'images')
        self.classes=[('Wear','Incorrect','Not Wear'),('남','여'),('<30','>=30 and <60','>=60')]
        self.trans=transform
        
        if self.train:
            self.criteria=1*(self.meta['age']>=30)+1*(self.meta['age']>=60)+3*(self.meta['gender']=='female')
            self.train_data,self.val_data=train_test_split(self.meta,test_size=0.2,stratify=self.criteria)
            self.data_dict={'train':self.train_data,'valid':self.val_data}
            self.data=make_images(self.data_dict[self.mod],self.img_dir,self.train)
        else:
            self.data=make_images(self.meta,self.img_dir,self.train)
    
    def __len__(self):
        return len(self.data)
        
    
    def __getitem__(self, idx):
        images=self.data['images']
        labels=self.data['labels']

        label=labels.iloc[idx]
        image=Image.open(images.iloc[idx])
        
        if self.mod=='train':
            if label%3==2:
                image=self.trans['uptrans'](image)
                return image,label
            else:
                image=self.trans['regtrans'](image)
        else:
            image=self.trans['trans'](image)
        return image,label
def conf_mat(y_true,y_pred):
    cm=confusion_matrix(y_true,y_pred)
    norm_cm=cm/np.sum(cm, axis=1)[:,None]
    cm=pd.DataFrame(norm_cm,index=['mask<30','mask<>','mask60','inc<30','inc<>','inc60','nom<30','nom<>','nom60'],columns=['mask<30','mask45','mask60','inc<30','inc<>','inc60','nom<30','nom<>','nom60'])
    fig=plt.figure(figsize=(20,16))
    sns.heatmap(cm,annot=True)
    return fig


# 합친 모델 학습과 결과저장

In [ ]:
net_mask = timm.create_model('efficientnet_b3a', pretrained=True, num_classes=3)
net_gender = timm.create_model('efficientnet_b3a', pretrained=True, num_classes=2)
net_age = timm.create_model('efficientnet_b3a', pretrained=True, num_classes=3)

net_mask.to(device)
net_gender.to(device)
net_age.to(device)

print('done')

In [38]:
#모델 구축
class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.mask=net_mask
        self.gender=net_gender
        self.age=net_age
        
    def forward(self,x):
        mask=(nn.Softmax(dim=1)(self.mask(x))).view(x.size(0),3,1,1)
        gender=(nn.Softmax(dim=1)(self.gender(x))).view(x.size(0),1,2,1)
        age=(nn.Softmax(dim=1)(self.age(x))).view(x.size(0),1,1,3)
        return (mask*gender*age).view(x.size(0),18)
    
class AgePredictor(nn.Module):
    def __init__():
        super().__init__()
        self.stage1=nn.Sequential([
            nn.Linear(3,16),
            nn.BatchNorm1d(16)
            nn.ReLU(),
        ])
        self.stage2=nn.Sequential([
            nn.Linear(16,256),
            nn.BatchNorm1d(256)
            nn.ReLU(),
        ])
        self.stage3=nn.Sequential([
            nn.Linear(256,256),
            nn.BatchNorm1d(256)
            nn.ReLU(),
        ])
        self.stage4=nn.Sequential([
            nn.Linear(256,256),
            nn.BatchNorm1d(256)
            nn.ReLU(),
        ])
        self.stage5=nn.Sequential([
            nn.Linear(256,1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
        ])
        self.fc=nn.Linear(1024,53)
    
    def forward(self,x):
        out=self.stage1(x)
        out=self.stage2(out)
        out=self.stage3(out)
        out=self.stage4(out)
        out=self.stage5(out)
        out=self.fc(out)
        return out

class PredDist(nn.Module):
    def __init__():
        super().__init__()
        self.age0=Normal(mean=18
        self.age1=
        self.age2=
        

In [39]:
#모델 설정하기
target=MyModel()
model=AgePredictor()
target_id=sorted(os.listdir(target_dir))[-1]
target=torch.load(os.path.join(target_dir,target_id))
target.load_state_dict(target['model_state_dict'])
target.to(device)
optim=optm.Adam(model.parameters())
optim.load_state_dict(target['optimizer_state_dict'])
scheduler = optm.lr_scheduler.StepLR(optim, step_size=1, gamma=0.75)
epoch=target['epoch']

In [40]:
#학습 설정하기
num_epochs=20
batch_size=60
learning_rate=0.0005
best_f1=0.
stopper=0
weight=torch.tensor([18900/49410,18900/36900,18900/7470,18900/65880,18900/73530,18900/9810,18900/9882,18900/7380,18900/1494,18900/13176,18900/14706,18900/1962,18900/9882,18900/7380,18900/1494,18900/13176,18900/14706,18900/1962]).to(device)
loss_fn=nn.CrossEntropyLoss(weight=weight)
config={'epochs':num_epochs,'batch_size':batch_size,'learning_rate':learning_rate}

In [ ]:
now='Runned'+'_'.join(str(datetime.now().astimezone(KST))[10:16].split(':'))
base_dir=os.path.join('.',f'saved/{project_name}/{date}/{now}')
wandb_dir=base_dir
checkpoint_dir=''
model_dir=''
result_dir=''
dirs={'model':model_dir,'checkpoint':checkpoint_dir,'result':result_dir}
for pth in ['model','checkpoint','result']:
    dirs[pth]=os.path.join(base_dir,pth)
    if not os.path.isdir(dirs[pth]):
        os.makedirs(dirs[pth])


wandb.init(project=project_name,config=config,dir=wandb_dir)
for e in range(epoch,epoch+num_epochs):   
    for mod in ['test','train','valid',]:
        print('mod:',mod)
        data=ImageDataset(mod=mod)

        if mod=='train':
            dataloader=DataLoader(data,batch_size=batch_size,shuffle=True)
            model.train()
        else:
            submission = pd.read_csv(os.path.join(data.path[0], 'info.csv'))
            dataloader=DataLoader(data,shuffle=False)
            model.eval()

        with tqdm(dataloader) as pbar:
            running_loss=0.
            running_acc=0.
            tot_pred=torch.tensor([]).to(device)
            tot_label=torch.tensor([]).to(device)
            cm_pred=torch.tensor([]).to(device)
            cm_label=torch.tensor([]).to(device)
            for n,(image,label) in enumerate(pbar):
                image=image.to(device)
                label=label.to(device)
                
                logit=model(image)
                _,pred=torch.max(logit,1)
                
                if data.train:
                    loss=loss_fn(logit,label)
                    running_loss+=loss.item()*image.size(0)
                    running_acc+=torch.sum(pred==label)/image.size(0)

                if mod=='train':
                    optim.zero_grad()
                    loss.backward()
                    optim.step()                    

                tot_pred=torch.hstack((tot_pred,pred))
                tot_label=torch.hstack((tot_label,label))
                cm_pred=torch.hstack((cm_pred,pred+3*classes['mask']))
                cm_label=torch.hstack((cm_label,label+3*classes['mask']))
                if data.train:
                    pbar.set_postfix({'epoch' : e, 'loss' : running_loss/(n+1), 'accuracy' : float(running_acc)/(n+1),'F1 score':f1_score(tot_label.cpu(),tot_pred.cpu(),average='macro')})
                    if mod=='train':
                        wandb.log({f'running loss {e}' : running_loss/(n+1), f'running accuracy {e}' : float(running_acc)/(n+1),f'running f1 score {e}':f1_score(tot_label.cpu(),tot_pred.cpu(),average='macro')})
            if data.train:
                epoch_loss=running_loss/len(dataloader.dataset)
                epoch_acc=running_acc*image.size(0)/len(dataloader.dataset)
                epoch_f1=f1_score(tot_label.cpu(),tot_pred.cpu(),average='macro')
                print(f"현재 epoch-{e}의 평균 Loss : {epoch_loss:.3f}, 평균 Accuracy : {epoch_acc:.3f}, F1 score : {epoch_f1:.3f}")
            elif mod=='test':
                submission['ans'] = tot_pred.cpu().numpy().astype('int64')
                submission.to_csv(os.path.join(result_dir, f'submission_{project_name}_{e}_{epoch_f1}.csv'), index=False)
                
    if mod=='valid':
        now=str(datetime.now().astimezone(KST))[:16]
        cm=conf_mat(cm_label.cpu(),cm_pred.cpu())
        cm.savefig(f'./saved/{project_name}/checkpoint/checkpoint_{key}_{now}_{e}.jpg')
        wandb.log({'confusion_matrix':wandb.Image(cm),'loss' : epoch_loss, 'accuracy' : epoch_acc,'f1 score':f1_score(tot_label.cpu(),tot_pred.cpu(),average='macro')})
        if best_f1<epoch_f1:
            stopper=0
            best_f1=epoch_f1
            torch.save({'epoch':e,'loss':loss,'model_state_dict':model.state_dict(),'optimizer_state_dict':optim.state_dict()},f"{dirs['checkpoint'}/checkpoint_{e.zfill(3)}_{epoch_loss:.3f}_{epoch_acc:.3f}_{epoch_f1:.3f}.pt")
        elif stopper==3:
            break
        else:
            stopper+=1
wandb.finish()

wandb: wandb version 0.12.1 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
